In [4]:
import pandas as pd
import numpy as np
from os import path
import collections

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_path = '/content/drive/MyDrive/Ml/lab2/Data'

In [5]:
transactions = pd.read_csv(path.join(base_path, 'transactions.csv'), nrows=1000000)
tr_types = pd.read_csv(path.join(base_path, 'tr_types.csv'), sep=';')
gender_train = pd.read_csv(path.join(base_path, 'gender_train.csv'))
tr_mcc_codes = pd.read_csv(path.join(base_path, 'tr_mcc_codes.csv'), sep=';')

In [ ]:
print(transactions.head(1))
print(tr_types.head(1))
print(gender_train.head(1))
print(tr_mcc_codes.head(1))


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [8]:
sample_tr_types = transactions.tr_type.sample(1000).tolist()
sampled_tr_descriptions_counts = tr_types[tr_types.tr_type.isin(sample_tr_types)].tr_description.value_counts(normalize=True)
keys = sampled_tr_descriptions_counts.keys()
pattern = keys.str.contains('POS') | keys.str.contains('АТМ')
print(f'{sampled_tr_descriptions_counts[pattern].sum() * 100:.2f}%')

67.74%


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
transactions_frequency = transactions.tr_type.value_counts(normalize=True)
freq_keys = transactions_frequency.head(10).index
tr_types[tr_types.tr_type.isin(freq_keys)]

,tr_type,tr_description
39,7010,Взнос наличных через АТМ (в своем тер.банке)
47,7030,Перевод на карту (с карты) через АТМ (в предел...
59,7070,Перевод на карту (с карты) через Мобильный бан...
60,7071,Перевод на карту (с карты) через Мобильный бан...
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
106,2010,Выдача наличных в АТМ Сбербанк России
129,2370,Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [ ]:
income_transactions = transactions[transactions.amount > 0][["customer_id", "amount"]]
outcome_transactions = transactions[transactions.amount < 0][["customer_id", "amount"]]
outcome_transactions.amount = outcome_transactions.amount.abs()
client_income = income_transactions.groupby(by='customer_id').sum().sort_values(by='amount', ascending=False)
client_outcome = outcome_transactions.groupby(by='customer_id').sum().sort_values(by='amount', ascending=False)

In [ ]:
top_income_client_id = client_income.idxmax().item()
top_outcome_client_id = client_outcome.idxmax().item()
print(f'Top income client id:  {top_income_client_id}', f'Top outcome client id: {top_outcome_client_id}', sep='\n')

Top income client id:  70780820
Top outcome client id: 70780820


In [ ]:
def get_customer_transactions_sum(customer_id):
  return abs(transactions[transactions.customer_id == customer_id].amount.sum())

In [ ]:
print(f'Client: {top_income_client_id} diff: {get_customer_transactions_sum(top_income_client_id):.2f}')
print(f'Client: {top_outcome_client_id} diff: {get_customer_transactions_sum(top_outcome_client_id):.2f}')

Client: 70780820 diff: 1837317.98
Client: 70780820 diff: 1837317.98


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
transactions_top_ten = transactions[transactions.tr_type.isin(freq_keys)]
median = transactions_top_ten.groupby(by='tr_type').amount.median()
mean = transactions_top_ten.groupby(by='tr_type').amount.mean()
pd.concat([mean, median], axis=1, keys=['Mean', 'Median'])

,Mean,Median
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [ ]:
def get_customer_transactions_mean_median(customer_id):
  customer_transactions = transactions[transactions.customer_id == top_income_client_id]
  mean = customer_transactions.groupby(by='tr_type').amount.mean()
  median = customer_transactions.groupby(by='tr_type').amount.median()
  return pd.concat([mean, median], axis=1, keys=['Mean', 'Median'])

In [ ]:
print('top income')
get_customer_transactions_mean_median(top_income_client_id)

top income


,Mean,Median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


In [ ]:
print('top outcome')
get_customer_transactions_mean_median(top_outcome_client_id)

top outcome


,Mean,Median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

In [ ]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
gender_outcome = transactions[transactions.amount < 0].groupby(by='gender').amount.mean().abs()
gender_income = transactions[transactions.amount > 0].groupby(by='gender').amount.mean()
print(f'gender income diff:  {abs(gender_income[0] - gender_income[1]):.2f}')
print(f'gender outcome diff: {abs(gender_outcome[0] - gender_outcome[1]):.2f}')

gender income diff:  63366.57
gender outcome diff: 32718.05


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
max_income = transactions[transactions['amount'] > 0].groupby(by=['gender', 'tr_type']).amount.sum().reset_index()
top_ten_gender_income = max_income.groupby('gender').apply(lambda x: x.sort_values(by='amount').head(10))
top_ten_gender_income

gender  tr_type     amount
gender                               
0.0    10     0.0     4051    1122.96
       13     0.0     4210    2245.92
       12     0.0     4110    2245.92
       9      0.0     2370    2335.75
       36     0.0     7075    6737.75
       6      0.0     2110   22459.16
       0      0.0     1010   31532.66
       11     0.0     4100   40334.88
       7      0.0     2210   68315.82
       4      0.0     1210   83644.87
1.0    44     1.0     2020    2245.92
       47     1.0     4110    4491.84
       38     1.0     1010   35317.03
       71     1.0     8100   64682.37
       48     1.0     6000   79280.83
       66     1.0     7041   87590.72
       46     1.0     2370  113194.15
       57     1.0     7015  121279.45
       45     1.0     2110  179673.26
       43     1.0     2010  224591.59

In [ ]:
zeroe_cats = set(top_ten_gender_income[top_ten_gender_income.gender == 0].tr_type.values)
one_cats = set(top_ten_gender_income[top_ten_gender_income.gender == 1].tr_type.values)
zeroe_cats.intersection(one_cats)

{1010, 2110, 2370, 4110}

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
sum_outcome = transactions[transactions['amount'] < 0].groupby(['gender', 'mcc_code', 'mcc_description']).amount.sum().abs()

merged = sum_outcome[0].reset_index().merge(sum_outcome[1].reset_index(), on='mcc_code', how='inner')
merged['diff'] = abs(merged['amount_x'] - merged['amount_y'])
merged.sort_values(by='diff', ascending=False)[['mcc_code', 'mcc_description_x', 'diff']].head(10)

,mcc_code,mcc_description_x,diff
116,6011,Финансовые институты — снятие наличности автом...,3.085862e+09
21,4829,Денежные переводы,2.499153e+09
115,6010,Финансовые институты — снятие наличности вручную,2.208580e+08
62,5541,Станции техобслуживания,1.554326e+08
59,5511,"Легковой и грузовой транспорт — продажа, серви...",7.352877e+07
85,5812,"Места общественного питания, рестораны",7.037856e+07
108,5977,Магазины косметики,4.241762e+07
61,5533,Автозапчасти и аксессуары,4.233808e+07
148,7995,Транзакции по азартным играм,4.079564e+07
71,5661,Обувные магазины,3.815470e+07


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
transactions[['tr_day', 'tr_time']] = transactions.tr_datetime.str.split(' ', expand=True)
transactions[['tr_hour', 'tr_minute', 'tr_sec']] = transactions.tr_time.str.split(':', expand=True);
transactions.tr_hour = pd.to_numeric(transactions.tr_hour)

In [ ]:
pattern = (transactions.tr_hour <= 6) & (transactions.tr_hour >= 0) & (transactions.amount < 0)
transactions[pattern].groupby('gender').amount.count()

gender
0.0    43510
1.0    46694
Name: amount, dtype: int64

 # Дополнительное задание

В датафрейм из задания 7.2 добавьте ещё один столбец с указанием пола, представители которого тратят по соответствующей категории больше. Сделайте так , чтобы в датафрейме было по 5 строк для каждого из двух полов.

In [ ]:
merged['max_gender'] = (merged['amount_x'] < merged['amount_y']).astype(int)
merged.groupby('max_gender').apply(lambda x: x.sort_values(by='diff', ascending=False)[['mcc_code', 'mcc_description_x', 'diff']].head(5))

mcc_code                                  mcc_description_x  \
max_gender                                                                    
0          108      5977                                 Магазины косметики   
           71       5661                                   Обувные магазины   
           66       5621                             Готовая женская одежда   
           89       5912                                             Аптеки   
           73       5691                  Магазины мужской и женской одежды   
1          116      6011  Финансовые институты — снятие наличности автом...   
           21       4829                                  Денежные переводы   
           115      6010   Финансовые институты — снятие наличности вручную   
           62       5541                            Станции техобслуживания   
           59       5511  Легковой и грузовой транспорт — продажа, серви...   

                        diff  
max_gender                    
0          108  4.241762e+07  
           71   3.815470e+07  
           66   3.804426e+07  
           89   3.641087e+07  
           73   3.307225e+07  
1          116  3.085862e+09  
           21   2.499153e+09  
           115  2.208580e+08  
           62   1.554326e+08  
           59   7.352877e+07